In [1]:
import os

In [2]:
!pwd

/Users/ricky/Desktop/Finally_Productive/end-to-end_1of3_ChestCancerTSClassification/End-to-End-CancerClassification/research


In [3]:
os.chdir("../")

In [4]:
!pwd

/Users/ricky/Desktop/Finally_Productive/end-to-end_1of3_ChestCancerTSClassification/End-to-End-CancerClassification


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# read yaml files
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
# Create Configuration Manager

class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # Read Yaml files from config and params
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Run the create directories python script
        create_directories([self.config.artifacts_root]) # creates artifacts/

    # Data Ingestions Configuration
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Run the create directories python script
        create_directories([config.root_dir]) # creates artifacts/data_ingestion/

        # Data Ingestions Config
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import gdown
import zipfile

from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Fetch and download data From URL
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            # define URL and make data directories
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True) # create folder if need be
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            # Extract file ID and Download the zip file
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            # Log successful Downlad
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            # Raise Exception
            raise e

        
    # Unzip Data
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        # Unzip and read
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True) # create folder if need be
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# Pipeline is the calling of all these methods

try:
    # define Configuration Manager
    config = ConfigurationManager()
    # Get config of data ingestions
    data_ingestion_config = config.get_data_ingestion_config()
    # define data ingestion object
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # download the data
    data_ingestion.download_file()
    # unzip the data
    data_ingestion.extract_zip_file()
except Exception as e:
    # Raise Exception
    raise e

[2025-04-24 09:24:04,523: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-24 09:24:04,526: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-24 09:24:04,527: INFO: common: created directory at: artifacts]
[2025-04-24 09:24:04,528: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-24 09:24:04,529: INFO: 2501569081: Downloading data from https://drive.google.com/file/d/1JTOSw3-wfeUuQHmNkxZCmKOJqt9Kkt3c/view?usp=share_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1JTOSw3-wfeUuQHmNkxZCmKOJqt9Kkt3c
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1JTOSw3-wfeUuQHmNkxZCmKOJqt9Kkt3c&confirm=t&uuid=47081388-85e1-4956-a159-13125df4853a
To: /Users/ricky/Desktop/Finally_Productive/end-to-end_1of3_ChestCancerTSClassification/End-to-End-CancerClassification/artifacts/data_ingestion/data.zip
100%|██████████| 125M/125M [00:16<00:00, 7.73MB/s] 

[2025-04-24 09:24:24,585: INFO: 2501569081: Downloaded data from https://drive.google.com/file/d/1JTOSw3-wfeUuQHmNkxZCmKOJqt9Kkt3c/view?usp=share_link into file artifacts/data_ingestion/data.zip]
